# General Prompt Engineering with GPT-4.1  
  
This notebook will help you master prompt engineering—crafting instructions to reliably get the best results from GPT-4.1 and related LLMs. We’ll use best practices and practical strategies for clarity, accuracy, and efficiency.  
  
---  
  
## 1️⃣ What is Prompt Engineering?  
  
**Prompt engineering** is the science of giving language models instructions to get high-quality outputs. Effective prompt engineering involves:  
- Giving clear, detailed instructions,  
- Supplying examples,  
- Adding context or references,  
- Specifying output format and priorities.  
  
By crafting better prompts, you can drastically improve response quality for any application—from Q&A, to code generation, to document analysis.  
  
---  

## 2️⃣ Chat Messages and Roles  
  
Unlike traditional models, GPT-4.1 and newer chat models take a structured **array of messages**. Each message has a **role** that guides the model’s interpretation:  
  
| Role       | Description                                                                |  
|------------|----------------------------------------------------------------------------|  
| developer  | High-priority “system” message, sets default behavior, tone, constraints   |  
| user       | The primary input or instruction from the end-user                         |  
| assistant  | (Optional) Previous responses or demonstrations                            |  
  
**Example: Persona and Style Control**  
  

In [ ]:
# Import libraries and load environment variables  
  
import os  
from openai import AzureOpenAI  
from dotenv import load_dotenv  
  
# Load Azure credentials from a .env file (you should have credentials.env in your directory)  
load_dotenv("credentials.env")  
  
# Create Azure OpenAI client  
client = AzureOpenAI(  
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-01-01-preview",  
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")  
)  
  
print("✅ Azure OpenAI client initialized.")  

In [ ]:
# Example: Setting up a "southern belle programmer" assistant persona  
  
messages = [  
    {  
        "role": "developer",  
        "content": [  
            { "type": "text", "text": "You are a helpful assistant that answers programming questions in the style of a southern belle from the southeast United States." }  
        ]  
    },  
    {  
        "role": "user",  
        "content": [  
            { "type": "text", "text": "Are semicolons optional in JavaScript?" }  
        ]  
    }  
]  
  
response = client.chat.completions.create(  
    model="gpt-4.1",  
    messages=messages  
)  
print(response.choices[0].message.content)  

**Try changing the content of the developer message above—watch how the assistant’s style and expertise changes!**  
  
---  

## 3️⃣ Strategy 1: Write Clear Instructions  
  
- **Be explicit:** The more detail you give, the less the model needs to guess.  
- **Set expectations:** Specify length, style, steps, or format—don't assume!  
- **Sample:**  
  
> “Summarize this article in three bullet points, then ask one insightful follow-up question.”  
  
### 👎 Vague prompt  
  

In [ ]:
vague_prompt = [  
    {"role": "user", "content": "Summarize this text: John bought apples and oranges at the market."}  
]  
resp = client.chat.completions.create(model="gpt-4.1", messages=vague_prompt)  
print(resp.choices[0].message.content)  

### 👍 Improved prompt  
  

In [ ]:
clear_prompt = [  
    {"role": "user", "content": "Summarize the following in one bullet point, and use friendly language:\nJohn bought apples and oranges at the market."}  
]  
resp = client.chat.completions.create(model="gpt-4.1", messages=clear_prompt)  
print(resp.choices[0].message.content)  

**Tip:**    
For complex requests, break them into lists or use headings to guide the model.  
  
---  

## 4️⃣ Strategy 2: Provide Reference Text and Retrieval Augmented Generation (RAG)  
  
**Why?**    
Language models can hallucinate facts, especially on niche or recent topics. By providing reference text (such as documents, snippets, or database results), you guide the model to use trustworthy information.  
  
**How to do it:**  
- Delimit your reference text clearly (use triple quotes, markdown, XML tags, etc.).  
- Instruct the model to answer **only** from the provided materials (or to cite them).  
- For longer references, mention which delimiters or citation markers to use.  
  
---  
  
**Example 1: Injecting Reference Information**  
  

In [ ]:
messages = [  
    {"role": "developer",   
     "content": "Use ONLY the provided text (inside triple quotes) to answer user questions. If you can’t find the answer, say so."},  
    {"role": "user",   
     "content": '''"""The warranty for Product A12 lasts 12 months from the date of purchase. Returns are possible within 30 days of delivery."""\nWhat is the warranty period for Product A12?'''}  
]  
  
resp = client.chat.completions.create(model="gpt-4.1", messages=messages)  
print(resp.choices[0].message.content)  

In [ ]:
#example 2 
messages = [  
    {"role": "developer",   
     "content": 'Use only the document delimited by triple quotes. Cite any answer with {"citation": ...}. If you cannot answer, say "Insufficient information."'},  
    {"role": "user",   
     "content": '''"""The event will be held on July 10, 2024 at the Grand Hall downtown."""\nQuestion: When and where is the event?'''}  
]  
  
resp = client.chat.completions.create(model="gpt-4.1", messages=messages)  
print(resp.choices[0].message.content)  

## 5️⃣ Strategy 4: Give the Model Time to “Think” (Chain-of-Thought Prompting)  
  
**Why?**    
If given space to “think out loud,” models can solve problems more reliably, especially in reasoning, math, and evaluations.  
  
**How to do it:**  
- Explicitly instruct the model to "reason step by step" or "show your work before answering."  
- You can also ask for a structured plan or interim analysis.  
  
---  
  
**Example: Chain of Thought for a Math Problem**  
  

In [ ]:
messages = [  
    {"role": "developer",   
     "content": "First, work out your solution step by step before you give the final answer."},  
    {"role": "user",   
     "content": "If Alice has 3 apples and gets 5 more, how many apples does she have in total?"}  
]  
resp = client.chat.completions.create(model="gpt-4.1", messages=messages)  
print(resp.choices[0].message.content)  

In [ ]:
#Ask for error-check or self-evaluation
   
messages = [  
    {"role": "developer",   
     "content": "Solve the problem step by step, then check if you made any mistakes before answering."},  
    {"role": "user",   
     "content": "Multiply 17 by 23."}  
]  
resp = client.chat.completions.create(model="gpt-4.1", messages=messages)  
print(resp.choices[0].message.content)  

  
---  
  
```markdown  
## 6️⃣ Strategy 5: Use External Tools and Function Calling  
  
**Why?**    
Language models have limitations: they may not have up-to-date or domain-specific knowledge, and are unreliable for math or code execution. Giving them access to tools (retrievers, code interpreters, APIs) allows more accurate, sophisticated responses.  
  
**How to do it:**  
- Use OpenAI’s function calling features to let the model call your functions/tools.  
- For accurate calculations, instruct the model to write code (e.g., in markdown code blocks) and (optionally) execute and resubmit results.  
  
---  
  
**Example: Simple Function Call (Pseudocode)**  
  

In [22]:
def factorial(n):  
    if n < 0:  
        return "Error: n must be a non-negative integer"  
    result = 1  
    for i in range(2, n+1):  
        result *= i  
    return result  

In [23]:
functions = [  
    {  
        "name": "factorial",  
        "description": "Calculates the factorial of a non-negative integer.",  
        "parameters": {  
            "type": "object",  
            "properties": {  
                "n": {"type": "integer", "description": "A non-negative integer"}  
            },  
            "required": ["n"]  
        }  
    }  
]  
messages = [  
    {"role": "user", "content": "What is the factorial of 5?"}  
]  

In [ ]:
#Call OpenAI and inspect the function call. The function call info is in response.choices[0].message.function_call, not in .message.content (which will be None for function calls)!
response = client.chat.completions.create(  
    model="gpt-4.1",  
    messages=messages,  
    functions=functions  
)  
  
# Extract function call  
function_call = response.choices[0].message.function_call  
print(function_call)  # See what the model wants to call  

In [ ]:
import json  
  
args = json.loads(function_call.arguments)  
result = factorial(args["n"])  
  
print(result)    # <- This shows the numerical result  

In [ ]:
messages.append({  
    "role": "function",  
    "name": "factorial",  
    "content": json.dumps({"result": result})  
})  
  
response2 = client.chat.completions.create(  
    model="gpt-4.1",  
    messages=messages  
)  
  
print(response2.choices[0].message.content)  
# Now you get "The factorial of 5 is 120." or similar  

# 📊 Strategy 7: Evaluation / Testing  
  
Test the model's responses by giving a prompt and comparing its answer against a gold-standard. We can measure similarity and set up grading.  
  
Below, we use a simple similarity check using Python's `difflib`. For robust setups, consider BLEU, ROUGE, or manual rubric scoring.  

In [ ]:
# Evaluation: Compare Model's Answer to Gold Standard  
  
from difflib import SequenceMatcher  
  
def evaluate_lm_response(gold_answer, model_response):  
    s = SequenceMatcher(None, gold_answer.strip().lower(), model_response.strip().lower())  
    similarity = s.ratio()  
    print(f"Similarity score: {similarity:.2f}")  
    return similarity  
  
# Example  
prompt = "What is the capital of France?"  
gold_answer = "The capital of France is Paris"  
model_response = "The capital of France is Paris."  
  
evaluate_lm_response(gold_answer, model_response)  

# 📋 General LLM Tactics (Reference Table)  
  
This table highlights prompt engineering and API usage tactics for optimizing LLM results.  
  
| Tactic        | Description                        | Example                                               |  
|---------------|------------------------------------|-------------------------------------------------------|  
| Delimiters    | Use ```, """, < > etc. to separate | `"""Summarize this passage: ..."""`                   |  
| Output Length | Limit/maximize response size       | `max_tokens=100` or "In 2 sentences or less..."       |  
| Few-shot      | Provide sample Q&A before query    | "Q: 2+2?\nA: 4\nQ: 3+5?\nA:"                         |  
| Personas      | Specify a role or identity         | "You are a helpful assistant."                        |  
| Summarization | Ask to summarize/shorten           | "Summarize the following text briefly."               |  
| Citations     | Request sources or quotes          | "List your sources at the end."                       |  

# ⚡ Optimizing for Accuracy, Cost, Latency  
  
Choose your settings and prompts to balance quality, speed, and price.  
  
**Examples:**  

In [ ]:
# Example 1: Limit cost by capping response length  
response = client.chat.completions.create(  
    model="gpt-4.1",  
    messages=[{"role": "user", "content": "Explain relativity simply."}],  
    max_tokens=50    # Lower token limit = lower cost  
)  
print(response.choices[0].message.content)  
  
# Example 2: Increase speed with smaller models or streaming  
response = client.chat.completions.create(  
    model="gpt-4.1",   
    messages=[{"role": "user", "content": "Summarize this article..."}],  
    stream=True   # Streams output for faster perceived response  
)  
for chunk in response:  
    if chunk.choices and hasattr(chunk.choices[0].delta, "content"):  
        print(chunk.choices[0].delta.content or "", end="")  
print()  # Newline after the streamed output   
  
# Example 3: Explicit instructions for higher accuracy  
prompt = (  
    "You are a medical expert. "  
    "In one sentence, explain what insulin does."  
)  
response = client.chat.completions.create(  
    model="gpt-4.1",  
    messages=[{"role": "user", "content": prompt}]  
)  
print(response.choices[0].message.content)  

# 📚 Additional Resources  
  
- OpenAI Cookbook : https://platform.openai.com/docs/guides/prompt-engineering/prompt-engineering